In [1]:
import os
from glob import glob
import numpy as np

from models.VariationalAutoEncoder import VariationalAutoEncoder
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.
/home/comadan/.venv/gdl/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Set Run Parameters

In [2]:
# run params
SECTION = 'vae'
RUN_ID = '0001'
DATA_NAME = 'faces'
if not os.path.exists("run"):
    os.mkdir("run")
if not os.path.exists(f"run/{SECTION}"):
    os.mkdir(f"run/{SECTION}")
RUN_FOLDER = f'run/{SECTION}/'
RUN_FOLDER += '_'.join([RUN_ID, DATA_NAME])

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

MODE =  'build' #'load' #

DATA_FOLDER = './data/celeb/'

## Load the data

In [3]:
input_dim = (128, 128, 3)
batch_size = 32
reconstruction_loss_multiplier = 10000

In [4]:
filenames = np.array(glob(os.path.join(DATA_FOLDER, '*/*.jpg')))

NUM_IMAGES = len(filenames)

In [5]:
data_gen = ImageDataGenerator(rescale=1./255)

data_flow = data_gen.flow_from_directory(DATA_FOLDER, 
                                         target_size = input_dim[:2], 
                                         batch_size = batch_size, 
                                         shuffle = True, 
                                         class_mode = 'input', 
                                         subset = "training")

Found 202599 images belonging to 1 classes.


## Define the structure of the neural network

In [6]:
encoder_architecture = [
    {'filter': 32, 'kernel': (3, 3), 'stride': 2},
    {'filter': 64, 'kernel': (3, 3), 'stride': 2},
    {'filter': 64, 'kernel': (3, 3), 'stride': 2},
    {'filter': 64, 'kernel': (3, 3), 'stride': 2},
    ]
decoder_architecture = [
    {'filter': 64, 'kernel': (3, 3), 'stride': 2},
    {'filter': 64, 'kernel': (3, 3), 'stride': 2},
    {'filter': 32, 'kernel': (3, 3), 'stride': 2},
    {'filter': 3, 'kernel': (3, 3), 'stride': 2},
    ]
latent_dim = 200
use_batch_norm = True
use_dropout = True

In [7]:
ae = VariationalAutoEncoder(
    input_dim=input_dim,
    latent_dim=latent_dim,
    encoder_params=encoder_architecture,
    decoder_params=decoder_architecture,
    use_batch_norm=use_batch_norm,
    use_dropout=use_dropout,
    reconstruction_loss_multiplier=reconstruction_loss_multiplier,
)












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



In [8]:
if MODE == 'build':
    ae.save(RUN_FOLDER)
else:
    ae.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))

In [9]:
ae.autoencoder_model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
encoder_conv_0 (Conv2D)         (None, 64, 64, 32)   896         encoder_input[0][0]              
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 64, 64, 32)   128         encoder_conv_0[0][0]             
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 64, 64, 32)   0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [10]:
ae.encoder_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
encoder_conv_0 (Conv2D)         (None, 64, 64, 32)   896         encoder_input[0][0]              
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 64, 64, 32)   128         encoder_conv_0[0][0]             
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 64, 64, 32)   0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [11]:
ae.decoder_model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              823296    
_________________________________________________________________
reshape_1 (Reshape)          (None, 8, 8, 64)          0         
_________________________________________________________________
decoder_conv_0 (Conv2DTransp (None, 16, 16, 64)        36928     
_________________________________________________________________
batch_normalization_5 (Batch (None, 16, 16, 64)        256       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 16, 16, 64)        0   

## Train the autoencoder

In [12]:
LEARNING_RATE = 0.0005
BATCH_SIZE = 32
INITIAL_EPOCH = 0
TOTAL_EPOCHS = 200
PRINT_EVERY_N_BATCHES = 100


In [13]:
ae.compile(LEARNING_RATE)

In [14]:
ae.train_with_generator(     
    data_flow
    , epochs = TOTAL_EPOCHS
    , steps_per_epoch = NUM_IMAGES / BATCH_SIZE
    , run_folder = RUN_FOLDER
    , print_every_n_batches = PRINT_EVERY_N_BATCHES
    , initial_epoch = INITIAL_EPOCH
)



Epoch 1/200
6332/6331 [==============================] - 264s 42ms/step - loss: 258.9143 - reconstruction_loss: 196.6346 - kl_divergence_loss: 62.2797

Epoch 00001: saving model to run/vae/0001_faces/weights/weights-001-258.90.h5

Epoch 00001: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 2/200
6332/6331 [==============================] - 266s 42ms/step - loss: 222.7037 - reconstruction_loss: 162.4357 - kl_divergence_loss: 60.2680

Epoch 00002: saving model to run/vae/0001_faces/weights/weights-002-222.69.h5

Epoch 00002: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 3/200
6332/6331 [==============================] - 260s 41ms/step - loss: 217.6187 - reconstruction_loss: 157.3857 - kl_divergence_loss: 60.2329

Epoch 00003: saving model to run/vae/0001_faces/weights/weights-003-217.61.h5

Epoch 00003: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 4/200
6332/6331 [==============================] - 257s 41ms/step - loss: 215.0937 - reconstructio

6332/6331 [==============================] - 258s 41ms/step - loss: 206.7719 - reconstruction_loss: 146.2149 - kl_divergence_loss: 60.5570

Epoch 00027: saving model to run/vae/0001_faces/weights/weights-027-206.77.h5

Epoch 00027: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 28/200
6332/6331 [==============================] - 260s 41ms/step - loss: 206.7136 - reconstruction_loss: 146.1706 - kl_divergence_loss: 60.5431

Epoch 00028: saving model to run/vae/0001_faces/weights/weights-028-206.71.h5

Epoch 00028: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 29/200
6332/6331 [==============================] - 258s 41ms/step - loss: 206.6194 - reconstruction_loss: 146.0683 - kl_divergence_loss: 60.5511

Epoch 00029: saving model to run/vae/0001_faces/weights/weights-029-206.62.h5

Epoch 00029: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 30/200
6332/6331 [==============================] - 260s 41ms/step - loss: 206.5654 - reconstruction_loss: 146

6332/6331 [==============================] - 258s 41ms/step - loss: 205.4445 - reconstruction_loss: 144.8478 - kl_divergence_loss: 60.5967

Epoch 00054: saving model to run/vae/0001_faces/weights/weights-054-205.44.h5

Epoch 00054: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 55/200
6332/6331 [==============================] - 259s 41ms/step - loss: 205.3170 - reconstruction_loss: 144.7631 - kl_divergence_loss: 60.5539

Epoch 00055: saving model to run/vae/0001_faces/weights/weights-055-205.31.h5

Epoch 00055: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 56/200
6332/6331 [==============================] - 259s 41ms/step - loss: 205.3460 - reconstruction_loss: 144.7822 - kl_divergence_loss: 60.5638

Epoch 00056: saving model to run/vae/0001_faces/weights/weights-056-205.34.h5

Epoch 00056: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 57/200
6332/6331 [==============================] - 261s 41ms/step - loss: 205.3476 - reconstruction_loss: 144

6332/6331 [==============================] - 260s 41ms/step - loss: 204.8384 - reconstruction_loss: 144.2705 - kl_divergence_loss: 60.5679

Epoch 00081: saving model to run/vae/0001_faces/weights/weights-081-204.84.h5

Epoch 00081: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 82/200
6332/6331 [==============================] - 258s 41ms/step - loss: 204.7715 - reconstruction_loss: 144.1993 - kl_divergence_loss: 60.5722

Epoch 00082: saving model to run/vae/0001_faces/weights/weights-082-204.76.h5

Epoch 00082: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 83/200
6332/6331 [==============================] - 259s 41ms/step - loss: 204.7573 - reconstruction_loss: 144.1928 - kl_divergence_loss: 60.5645

Epoch 00083: saving model to run/vae/0001_faces/weights/weights-083-204.76.h5

Epoch 00083: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 84/200
6332/6331 [==============================] - 259s 41ms/step - loss: 204.7630 - reconstruction_loss: 144

6332/6331 [==============================] - 257s 41ms/step - loss: 204.4104 - reconstruction_loss: 143.8324 - kl_divergence_loss: 60.5781

Epoch 00108: saving model to run/vae/0001_faces/weights/weights-108-204.41.h5

Epoch 00108: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 109/200
6332/6331 [==============================] - 261s 41ms/step - loss: 204.4504 - reconstruction_loss: 143.8445 - kl_divergence_loss: 60.6059

Epoch 00109: saving model to run/vae/0001_faces/weights/weights-109-204.45.h5

Epoch 00109: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 110/200
6332/6331 [==============================] - 261s 41ms/step - loss: 204.4074 - reconstruction_loss: 143.8279 - kl_divergence_loss: 60.5795

Epoch 00110: saving model to run/vae/0001_faces/weights/weights-110-204.41.h5

Epoch 00110: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 111/200
6332/6331 [==============================] - 259s 41ms/step - loss: 204.4179 - reconstruction_loss: 

6332/6331 [==============================] - 260s 41ms/step - loss: 204.2177 - reconstruction_loss: 143.6392 - kl_divergence_loss: 60.5785

Epoch 00135: saving model to run/vae/0001_faces/weights/weights-135-204.21.h5

Epoch 00135: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 136/200
6332/6331 [==============================] - 261s 41ms/step - loss: 204.2349 - reconstruction_loss: 143.6302 - kl_divergence_loss: 60.6047

Epoch 00136: saving model to run/vae/0001_faces/weights/weights-136-204.24.h5

Epoch 00136: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 137/200
6332/6331 [==============================] - 259s 41ms/step - loss: 204.1819 - reconstruction_loss: 143.5806 - kl_divergence_loss: 60.6013

Epoch 00137: saving model to run/vae/0001_faces/weights/weights-137-204.18.h5

Epoch 00137: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 138/200
6332/6331 [==============================] - 260s 41ms/step - loss: 204.1926 - reconstruction_loss: 

6332/6331 [==============================] - 260s 41ms/step - loss: 204.0861 - reconstruction_loss: 143.5090 - kl_divergence_loss: 60.5771

Epoch 00162: saving model to run/vae/0001_faces/weights/weights-162-204.08.h5

Epoch 00162: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 163/200
6332/6331 [==============================] - 261s 41ms/step - loss: 204.0870 - reconstruction_loss: 143.4660 - kl_divergence_loss: 60.6209

Epoch 00163: saving model to run/vae/0001_faces/weights/weights-163-204.08.h5

Epoch 00163: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 164/200
6332/6331 [==============================] - 260s 41ms/step - loss: 204.0559 - reconstruction_loss: 143.4572 - kl_divergence_loss: 60.5986

Epoch 00164: saving model to run/vae/0001_faces/weights/weights-164-204.05.h5

Epoch 00164: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 165/200
6332/6331 [==============================] - 260s 41ms/step - loss: 203.9770 - reconstruction_loss: 

6332/6331 [==============================] - 258s 41ms/step - loss: 203.9470 - reconstruction_loss: 143.3434 - kl_divergence_loss: 60.6036

Epoch 00189: saving model to run/vae/0001_faces/weights/weights-189-203.94.h5

Epoch 00189: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 190/200
6332/6331 [==============================] - 260s 41ms/step - loss: 203.8785 - reconstruction_loss: 143.2689 - kl_divergence_loss: 60.6096

Epoch 00190: saving model to run/vae/0001_faces/weights/weights-190-203.88.h5

Epoch 00190: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 191/200
6332/6331 [==============================] - 261s 41ms/step - loss: 203.8642 - reconstruction_loss: 143.2793 - kl_divergence_loss: 60.5850

Epoch 00191: saving model to run/vae/0001_faces/weights/weights-191-203.87.h5

Epoch 00191: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 192/200
6332/6331 [==============================] - 261s 41ms/step - loss: 203.8874 - reconstruction_loss: 